In [1]:
import tempfile
import pandas as pd
import rasterio as rt
import matplotlib.pyplot as plt
import numpy as np
import fiona
import rasterio.mask
from rasterio.plot import show
from rasterio.io import MemoryFile
import geopandas as gpd
import shapely
from shapely.geometry import Polygon
import cv2
from skimage import data
from skimage.filters import threshold_multiotsu
from scipy import stats
import subprocess
import os
import scipy.ndimage

In [5]:
def CC(img,dem,shp):
#     img = "G3_CC/2019_08_29_11_53/02_Mosaic_Project/RGB.tif"
#         shp = "G3_CC/05_ROI/PLOTS.shp"
#         dem = "G3_CC/2019_08_29_11_53/02_Mosaic_Project/DEMO.tif"
    dem = rt.open(dem)
    src = rt.open(img)## read img
    
    np.seterr(divide='ignore', invalid='ignore')
    blue = src.read(3).astype(float)
    green = src.read(2).astype(float)
    red = src.read(1).astype(float)
    VI = ((green - red) + (green - blue)) / ((2 * green) + red + blue)###GLI
#     VI = (green - red) / (green + red) #create VI visible

    out_meta = src.meta
    out_meta.update(
        dtype=rt.float32,
        count = 1)
    path = 'temp/GLI.tif'
    with rt.open(path, "w", **out_meta) as dest:
        dest.write_band(1, VI.astype(rt.float32))## write img

    def getFeatures(gdf,clumn_label,label): #create function for select ROI for ID and transform to GeoJson
        import json
        new = gdf[gdf[clumn_label]==label]
        new = new.dissolve(by=clumn_label)
        return [json.loads(new.to_json())['features'][0]["geometry"]]

    roi = gpd.read_file(shp)
    VI = rt.open(path)
    alldf = pd.DataFrame()
    for ID in roi.ID:
#         ID = 662
        path_vi = 'temp/mask_VI.tif'
        path_dem = 'temp/mask_dem.tif'
        path_mask_dem = 'temp/mask_th.tif'
        path_mask_dem_rs = 'temp/mask_th_resample.tif'
        paths = [path_vi,path_dem,path_mask_dem,path_mask_dem_rs]
        for i in paths:
            if os.path.exists(i):
                os.remove(i)
        coords = getFeatures(roi,"ID",ID) #run fuction extrac ID from ROI
        ############mask IMG
        out_image, out_transform = rt.mask.mask(VI, coords, crop=True) #extrac by mask
        out_meta = VI.meta
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
        out_image[out_image==0] = np.nan

        with rt.open(path_vi, "w", **out_meta) as dest:## write img
            dest.write(out_image)
        #############mask DEM
        out_image, out_transform = rt.mask.mask(dem, coords, crop=True)
        out_meta = dem.meta
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
        with rasterio.open(path_dem, "w", **out_meta) as dest:
            dest.write(out_image)
        # # ##################   Create threshold img from VI
        clipped_img = rt.open(path_vi)
        Img = clipped_img.read(1)#img to array
        mean_img = np.nanmean(Img)
        max_img = np.nanmax(Img)
        ################### Create threshold DEM
        clipped_dem = rasterio.open(path_dem)
        Dem = clipped_dem.read(1)
        Dem[Dem==clipped_dem.nodatavals[0]]=np.nan
#         std_dem = np.nanstd(Dem)
#         mean_dem = np.nanmean(Dem)
        min_dem = np.nanmin(Dem)
        max_dem = np.nanmax(Dem)
        delta = ((max_dem-min_dem)/3)*2.5
        th = max_dem - delta
        if mean_img<=0.1:
            ret, bin_img = cv2.threshold(Img, 0.035, 1,  cv2.THRESH_BINARY)#GLI SOIL 0.2 NGRDI 0 SOIL
            mask = bin_img
        elif mean_img>0.1 and mean_img<=0.385:
            ret, bin_img = cv2.threshold(Img, 0.035, 1,  cv2.THRESH_BINARY)

            ret, bin_dem = cv2.threshold(Dem, th, 1,  cv2.THRESH_BINARY)
            clipped_dem = rt.open(path_dem)
            out_meta = clipped_dem.meta
            out_meta.update({"driver": "GTiff",
                             'nodata': 0,
                             "height": bin_dem.shape[0],
                             "width": bin_dem.shape[1],
                             "dtype":rt.int8})
            with rasterio.open(path_mask_dem, "w", **out_meta) as dest:
                dest.write_band(1, bin_dem.astype(rt.int8))
            args = [
                'gdalwarp',
                '-tr', str(clipped_img.transform[0]), str(-clipped_img.transform[4]),
                '-r', 'bilinear',
                path_mask_dem, path_mask_dem_rs]
            subprocess.call(args)
            resample = rt.open(path_mask_dem_rs)
            img_rs = resample.read(1)
            if img_rs.shape == bin_img.shape:
                mask = img_rs*bin_img
            else:
                row = img_rs.shape[0] - bin_img.shape[0]
                col = img_rs.shape[1] - bin_img.shape[1]
                if row>=1:
                    img_rs = np.delete(img_rs, np.s_[0:row], axis=0)
                if col>=1:
                    img_rs = np.delete(img_rs, np.s_[0:col], axis=1)
                if row<=-1:
                    bin_img = np.delete(bin_img, np.s_[0:abs(row)], axis=0)
                if col<=-1:
                    bin_img = np.delete(bin_img, np.s_[0:abs(col)], axis=1)
                mask = img_rs*bin_img
            del(resample)
        else:
            ret, bin_img = cv2.threshold(Img, 0.035, 1,  cv2.THRESH_BINARY)
            mask = bin_img
        clipped_img = rt.open(path_vi)
        Img = clipped_img.read(1)
        Img[Img==clipped_img.nodatavals[0]]=np.nan
        Img[Img>=np.nanmin(Img)]=1
        out_meta = clipped_img.meta
        gt = clipped_img.transform
        pixelSizeX = gt[0]
        pixelSizeY =-gt[4]
        a = np.nansum(Img)*pixelSizeX*pixelSizeY
        b = pixelSizeX*pixelSizeY*np.nansum(mask)
        c = (b*100)/a
        del(clipped_img,clipped_dem)
        alldf = alldf.append({"Plots" : ID, "Area%" : round(c,3)}, ignore_index=True)
#     "mean_img": mean_img,"mean_dem": mean_dem,"std_dem":std_dem,"min_dem": min_dem,"max_dem":max_dem,"th_dem": th
    return alldf

In [6]:
from pathlib import Path
from itertools import chain
path = "G3_CC"
ext = "tif"
extention = r"{}".format(ext)
direction = Path(r"{}".format(path))
img_list = list(direction.glob(r"**/**/RGB.{}*".format(extention)))
dem_list = list(direction.glob(r"**/**/DEMO.{}*".format(extention)))
# dsm_list = list(direction.glob(r"**/**/DSM.{}*".format(extention)))
# print(len(img_list[:4]))
# len(dem_list[:4])
dem_list
# img_list

[WindowsPath('G3_CC/2019_04_17_11_35/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_06_04_13_39/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_06_21_14_04/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_07_16_15_15f/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_08_29_11_53/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_09_04_10_25/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_09_13_10_30/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_09_26_13_51/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_10_10_14_14/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_10_31_10_44/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_11_07_11_33/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2019_12_04_11_02/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2020_01_15_08_58/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2020_01_23_10_33/02_Mosaic_Project/DEMO.tif'),
 WindowsPath('G3_CC/2020_02_10_10_30/02_Mosaic_Project/DEMO.t

In [4]:
img_list

[WindowsPath('G3_CC/2019_04_17_11_35/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_06_04_13_39/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_06_21_14_04/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_07_16_15_15f/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_08_29_11_53/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_09_04_10_25/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_09_13_10_30/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_09_26_13_51/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_10_10_14_14/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_10_31_10_44/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_11_07_11_33/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2019_12_04_11_02/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2020_01_15_08_58/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2020_01_23_10_33/02_Mosaic_Project/RGB.tif'),
 WindowsPath('G3_CC/2020_02_10_10_30/02_Mosaic_Project/RGB.tif')]

In [7]:
import timeit
start = timeit.timeit()
for i in zip(img_list[2],dem_list[2]):
    cc = CC(img = i[0],dem = i[1], shp = "G3_CC/05_ROI/PLOTS.shp")
    cc["Date"] = str(i)[20:30]
    path = "index/cc_"+str(i)[20:30]+"GLI.csv"
    cc.to_csv(path,index=False)
    print(i)
end = timeit.timeit()
print(end - start)

TypeError: 'WindowsPath' object is not iterable

In [15]:
files = os.listdir('index')
alldf = pd.DataFrame()
for file in files:
    df = pd.read_csv("index/"+file)
    df = df[["Area%","Plots","Date"]]
    alldf = pd.concat([alldf,df],ignore_index=True)
alldf.to_csv("index/alldf_new.csv")

NameError: name 'i' is not defined

In [60]:
cc = CC(img = img_list[1],dem = dem_list[1], shp = "G3_CC/05_ROI/PLOTS.shp")
cc["Date"] = str(img_list[1])[6:16]
path = "index/cc_"+str(img_list[1])[6:16]+"GLI.csv"
cc.to_csv(path,index=False)

In [58]:
shp = "G3_CC/05_ROI/PLOTS.shp"
path = img_list[1]
VI = rt.open(path)
roi = gpd.read_file(shp)
roi.crs

<Projected CRS: PROJCS["MAGNA_SIRGAS_Colombia_West_zone",GEOGCS["G ...>
Name: MAGNA_SIRGAS_Colombia_West_zone
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: Marco Geocentrico Nacional de Referencia
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [59]:
str(VI.crs)
roi = roi.to_crs(str(VI.crs))
roi.crs

<Projected CRS: EPSG:3115>
Name: MAGNA-SIRGAS / Colombia West zone
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Colombia - 78°35'W to 75°35'W
- bounds: (-78.59, 0.03, -75.58, 10.21)
Coordinate Operation:
- name: Colombia MAGNA West zone
- method: Transverse Mercator
Datum: Marco Geocentrico Nacional de Referencia
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich